In [11]:
import docx
import re
import os
from pathlib import Path
import win32com.client
import sys
from tqdm import tqdm
import pandas as pd
import helpers as hp

In [12]:
def storeInfo(f, tags, df):
    for tag in list(tags.keys()):
        temp_tag = re.findall("«([^»]*)", tag)
        f.write(temp_tag[0] + "\n")
        # df[tag] = True
        df.loc[len(df.index)-1, temp_tag[0]] = True
    return df

In [13]:
def getTags(paths, rm=False):
    word = win32com.client.Dispatch("Word.Application")
    wdFormatDocumentDefault = 16

    df = pd.DataFrame()

    if paths['bulk']:
        # df = pd.DataFrame()
        for filepath in tqdm(sorted(Path(paths['input']).glob("*.doc"))):
            # if str(filepath).endswith(".doc"):
            doc_file = filepath.parent / filepath.stem
            # print(doc_file, ".docx")
            # document = word.Documents.Open(str(filepath))
            # document.SaveAs(str(doc_file) + ".docx", FileFormat=wdFormatDocumentDefault)
            # document.Close(0)
            hp.saveAsDocx(word, filepath)
            if rm:
                os.remove(str(doc_file) + ".doc")
        
        f = open('Info.txt', "w+")
        for filepath in tqdm(sorted(Path(paths['input']).glob("*.docx"))):
            df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
            # print(filepath)
            f.write("\n---- " + str(filepath.stem) + " ----\n")
            document = docx.Document(str(filepath))

            tags = {}
            tags = hp.extractTags(document, tags)
            # for line in document.paragraphs:
            #     # print(line.text)
            #     # word_lst = []
            #     for word in line.text.split():
            #         # word_lst.append(word)
            #         # word_lst.extend(re.findall("«.*»", word))

            #         temp = re.findall("«.*»", word)
            #         # print(word, temp)
            #         try:
            #             tags[temp[0]] = tags.get(temp[0], 0) + 1
            #         except:
            #             # print('---ERROR---')
            #             pass
                
                # print(word_lst)
            # print(tags)
            df = storeInfo(f, tags, df)
            # if rm:
            #     os.remove(str(filepath))
            # break
        f.close()
        # return df

    else:
        # df = pd.DataFrame()
        filepath = Path(paths['input'])
        # print(str(filepath))
        if str(filepath).endswith(".doc") or str(filepath).endswith(".DOC"):
            # print("Here")
            # document = word.Documents.Open(str(filepath))
            # document.SaveAs(str(filepath.parent / filepath.stem) + ".docx", FileFormat=wdFormatDocumentDefault)
            # document.Close(0)
            hp.saveAsDocx(word, filepath)
            if rm:
                os.remove(str(filepath.parent / filepath.stem) + ".doc")
        
        f = open('Info.txt', "w+")
        df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
        f.write("\n---- " + str(filepath.stem) + " ----\n")
        document = docx.Document(str(filepath.parent / filepath.stem) + ".docx")

        tags = {}
        tags = hp.extractTags(document, tags)
        # for line in document.paragraphs:
        #     for word  in line.text.split():
        #         temp = re.findall("«.*»", word)
        #         try:
        #             tags[temp[0]] = tags.get(temp[0], 0) + 1
        #         except:
        #             pass
        
        df = storeInfo(f, tags, df)
        # if rm:
        #         os.remove(str(filepath.parent / filepath.stem) + ".docx")
        f.close()
        # return df
    return df

In [10]:
inpath = 'D:\CCS\Template Attachments\myfolder'
paths = hp.resolvePath(inpath)
df = getTags(paths)

Identified input as a folder
{'bulk': True, 'input': 'D:\\CCS\\Template Attachments\\myfolder', 'output': 'D:\\CCS\\Template Attachments\\myfolder'}


In [22]:
df.fillna(False, inplace=True)
df.head()

,Name,MAILING_DATE_MONTH_X,MAILING_DATE_DAY_X,MAILING_DATE_YEAR_X,Customer_Name_X,Customer_Address_1_X,Customer_Address_2_X,Customer_Address_City_X,CUSTOMER_ADDRESS_STATE_X,CUSTOMER_ADDRESS_ZIP_X,Premise_Street_Address_X,MEMO_1_X,MEMO_2_X,MEMO_3_X,MEMO_4_X,PREFERRED_SIGNER_NAME_X,SIGNER_TITLE_X,SPANISH_SIGNATURE_TITLE_X
0,CCSE-121-UG Lateral Hardening_Mid Construction...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False
1,CCSE-122-UG Lateral Hardening_Project Complete...,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True
2,CCSE-123-UG Lateral Hardening_Project Complete...,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,False


In [9]:
df.to_csv('Template_Infos.csv', index=False)